# Colab-APPRAISE notebook


This is a note book that allows performing APPRAISE pipeline on Google Colab step-by-step.

Hit the play button on the left of each cell sequentially to run each block individually. You may expand the collapsed (hidden) cells for more settings and options. 

All results are saved in your Google Drive after each step. You can safely terminate a session between steps, although you'll need to re-run step 0 every time you restart.

Author: Xiaozhe Ding (Email: xding@caltech.edu, dingxiaozhe@gmail.com; Twitter: [@DingXiaozhe](https://twitter.com/dingxiaozhe?lang=en))

If you have any questions when using the notebook, please feel free to raise an issue in Github or shoot an email to me.

### Step 0 - Prepare environment (run everytime you restart a new session)

In [ ]:
#@title ####0.1 Mount google drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ####0.2 Install APPRAISE package

!pip install appraise

### Step 1 Prepare input files for competitive structure modeling

In [ ]:
import appraise
from appraise.utilities import *
from appraise.input_fasta_prep import *
import re

def check_alphanumeric(string):
    return bool(re.search("^[a-zA-Z0-9]+$", string))

##################################################
#@markdown ####**Basic settings**
##################################################

csv_file_path = '/content/drive/MyDrive/APPRAISE_project_1/input_csv/AAV_mock_selection_100_peptide_list.csv' #@param {type:"string"}
#@markdown - The .csv spreadsheet containing names and sequences of candidate engineered proteins.
#@markdown - The spreadsheet should contain at least two columns titled "peptide_name" and "peptide_seq", respectively. 
#@markdown - You may check [an example csv file](https://github.com/xz-ding/APPRAISE/blob/main/demo/example_input_sequences_from_manuscript/AAV_22x22/AAV_22x22_peptide_list.csv) provided in the [demo](https://github.com/xz-ding/APPRAISE/tree/main/demo/example_input_sequences_from_manuscript).

folder_path_for_fastas = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_input_fastas' #@param {type:"string"}
#@markdown - The destination folder that will contain the fasta files for structure modeling.

receptor_name = 'Ly6a' #@param {type:"string"}
#@markdown - Name of the receptor. **Use letters and numbers only.**
if check_alphanumeric(receptor_name) == False:
    raise ValueError("receptor_name does not allow non-alphanumeric characters. Please remove the special characters.")


receptor_seq = "LECYQCYGVPFETSCPSITCPYPDGVCVTQEAAVIVDSQTRKVKNNLCLPICPPNIESMEILGTKVNVKTSCCQEDLCNVAVP" #@param {type:"string"}
#@markdown - Receptor sequence to be used for modeling. 
#@markdown - We recommend using the minimal self-folding domain that is essential for the binding interaction to achieve the highest speed and accuracy.
#@markdown - Disordered sequences that are non-essential for binding may decrease the accuracy of modeling and should be truncated. Tips: 1. Truncated, crystallographic constructs used by structural biologists are usually good for modeling, if an experimental structure of the receptor is available; 2. You can also model the full-length receptor sequence using [ColabFold](https://github.com/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) and find out the low-confidence (e.g., pLDDT < 50) terminal sequences that may be truncated.
#@markdown - Example sequences can be found in the [demo](https://github.com/xz-ding/APPRAISE/blob/main/demo/example_input_sequences_from_manuscript/receptor_sequences.csv).

competition_mode = 'pooled' #@param ['pairwise', 'pooled', 'single']{type:"string"}
#@markdown - Modes of competition:
#@markdown -- **pairwise** Two candidate engineered proteins compete for a receptor. All possible pairs will be modeled exhaustively.
#@markdown -- **pooled** Candidate proteins will be grouped into fixed-sized groups and compete within the group. Each candidate protein will be in one group only. 
#@markdown -- **single** Complex models of one single engineered protein and the receptor will be modeled.


##################################################
#@markdown ####**Options for pairwise competition**
##################################################
square_matrix = True #@param {type:"boolean"}
#@markdown - If True, all peptides in the list will be competing with each oter through a complete tournament. 
#@markdown - If False, each peptide from the csv file above will be competing with each peptide from another spreadsheet. You'll need to provide the path to other .csv spreadsheet.
csv_file_path_2 = 'N/A' #@param {type:"string"}
#@markdown - Provide path to the additional .csv spreadsheet if square_matrix is False.


##################################################
#@markdown ####**Options for pooled competition**
##################################################
pool_size = '4' #@param ['2', '3', '4'] {type:"string"}
pool_size = int(pool_size)
#@markdown - The number of candidates in each pool. 
random_seed = 42 #@param{type:'integer'}
#@markdown - The seed for random grouping. The same seed number will result in the same grouping.
number_of_groupings = 2 #@param{type:'integer'}
#@markdown - It is recommended to model with multiple groupings to reduce the stochasticity of results. Total computational cost will scale linearly with the number of groupsings. 
#@markdown - Fasta files with different groupings will be merged in the same folder.

##################################################
#@markdown ####**Advanced options**
##################################################

use_glycine_linker = True #@param {type:"boolean"}
#@markdown - If True, the complex models will be modeled as a single chain protein joint with glycines.
#@markdown - Use this option if you intend to predict the structures with **ESMFold**.

glycine_linker_length=30 #@param {type:"integer"}
#@markdown - If use_glycine_linker is True, use this variable to adjust the length of glycine linker.

prepare_receptor_model = True #@param {type:"boolean"}
#@markdown - If True, a single-chain model with only the receptor will be modeled. The modeled structure can later be uploaded to [HullRad server](http://52.14.70.9/Run_hullrad.html) to measure the geometry properties of the receptor.

peptide_names, peptide_seqs = load_peptides(csv_file_path)
peptide_names_2 = []
peptide_seqs_2 = []
if competition_mode == 'pairwise' and square_matrix == 'False':
    peptide_names_2, peptide_seqs_2 = load_peptides(csv_file_path_2)

if competition_mode == 'pooled':
    loop_number = number_of_groupings
else:
    loop_number = 1

for i in range(loop_number):
    list_query_sequence, list_jobname = get_complex_fastas(receptor_name=receptor_name, receptor_seq=receptor_seq, list_peptide1_names=peptide_names,\
                                        list_peptide1_seqs=peptide_seqs, mode=competition_mode, square_matrix=square_matrix,\
                                        list_peptide2_names=peptide_names_2, list_peptide2_seqs=peptide_seqs_2, pool_size=pool_size,\
                                        folder_path=folder_path_for_fastas, use_glycine_linker=use_glycine_linker,\
                                        glycine_linker_length=glycine_linker_length, random_seed=random_seed,\
                                        prepare_receptor_model=prepare_receptor_model)
    random_seed += 1
    prepare_receptor_model = False

### Step 2 - Run structure prediction

**Instruction** 


The structures can be modeled using AlphaFold2-multimer or other state-of-the-art structure-prediction tools. 

For easy access to AlphaFold2-multimer, we suggest using ColabFold, an integrated implementation of multiple sequence alignment and AlphaFold (Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S and Steinegger M. ColabFold: Making protein folding accessible to all., Nature Methods (2022) doi: 10.1038/s41592-022-01488-1). The latest version of ColabFold can be found in its [GitHub repository sokrypton/ColabFold](https://github.com/sokrypton/ColabFold)

For easy execution, slightly-modified versions of ColabFold for batch execution of AlphaFold-multimer or ESMFold have been integrated below. You may choose one of them to proceed.

#### Step 2A - Predict structures with AlphaFold-multimer

Use this block if you want to perform the structure modeling using AlphaFold-multimer. 

This part of the notebook was modified from [ColabFold/AlphaFold2_batch.ipynb](https://github.com/sokrypton/ColabFold/blob/main/batch/AlphaFold2_batch.ipynb). We recommend using the default settings pre-filled below (execpt the directories). You can find more instructions in the original notebook.

In [ ]:
#@title #####2A.0 Show current GPU
#@markdown - Structure modeling will require using a GPU session. 
#@markdown - If you see "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver.", it means that your session is not equipped with a GPU accelarator. You may go to Colab's menu "Runtime --> Change runtime type --> Hardware accelerator" and choose "GPU". 
#@markdown - You'll need to re-run Step 0 after you change the runtime type.
#@markdown - Note: Different GPU models may result in slightly different prediction results.
!nvidia-smi

In [ ]:
#@title #####2A.1 Input protein sequence

from sys import version_info 
python_version = f"{version_info.major}.{version_info.minor}"

input_dir = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_input_fastas' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_results' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
model_type = "auto" #@param ["auto", "AlphaFold2-multimer-v2", "AlphaFold2-multimer-v1"] {type:"string"}
#@markdown - If "auto", AlphaFold2-multimer-v2 will be used for complexes and AlphaFold2 will be used for single chains.
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 5 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
use_amber = False #@param {type:"boolean"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False

In [ ]:

#@title #####2A.2 Install dependencies
%%bash -s use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  # high risk high gain
  pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" -f https://storage.googleapis.com/jax-releases/jax_releases.html
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi
     

In [ ]:
#@title ##### 2A.3 Run Prediction


# Force to download for the appropriate model type
if model_type == "AlphaFold2-multimer-v1":
    from colabfold.download import download_alphafold_params
    from pathlib import Path
    download_alphafold_params(model_type, Path("/root/.cache/colabfold/"))

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

if 'logging_setup' not in globals():
    setup_logging(Path(result_dir).joinpath("log.txt"))
    logging_setup = True

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    use_amber=use_amber,
    msa_mode=msa_mode,
    model_type=model_type,
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[3, 4, 5, 1, 2],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
)
     

In [ ]:
#@title ##### 2A.4 (optional) Terminate runtime
#@markdown The session will be terminated and you will see an "error message".


from google.colab import runtime
runtime.unassign()

#### Step 2B - Predict structures with ESMFold

Use this block if you want to perform the structure modeling using ESMFold. 

This part of the notebook was modified from [ColabFold/AlphaFold2_batch.ipynb](https://github.com/sokrypton/ColabFold/blob/main/batch/AlphaFold2_batch.ipynb) and [ColabFold/ESMFold.ipynb](https://github.com/sokrypton/ColabFold/blob/main/ESMFold.ipynb). We recommend using the default settings pre-filled below (execpt the directories). You can find more instructions in the original notebooks.

In [ ]:
#@title #####2B.0 Show current GPU
#@markdown - Structure modeling will require using a GPU session. 
#@markdown - If you see "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver.", it means that your session is not equipped with a GPU accelarator. You may go to Colab's menu "Runtime --> Change runtime type --> Hardware accelerator" and choose "GPU". 
#@markdown - You'll need to re-run Step 0 after you change the runtime type.
#@markdown - Note: Different GPU models may result in slightly different prediction results.
!nvidia-smi

In [ ]:
%%time
#@title #####2B.1 Installation
#@markdown install ESMFold, OpenFold, download Params (~2min 30s), and download other utilities from ColabFold

#Installation
import os, time
if not os.path.isfile("esmfold.model"):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/esmfold.model &")

  # install libs
  os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol")
  os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

  # install openfold
  commit = "6908936b68ae89f67755240e2f588c09ec31d4c8"
  os.system(f"pip install -q git+https://github.com/aqlaboratory/openfold.git@{commit}")

  # install esmfold
  os.system(f"pip install -q git+https://github.com/sokrypton/esm.git@beta")

  # wait for Params to finish downloading...
  if not os.path.isfile("esmfold.model"):
    # backup source!
    os.system("aria2c -q -x 16 https://files.ipd.uw.edu/pub/esmfold/esmfold.model")
  else:
    while os.path.isfile("esmfold.model.aria2"):
      time.sleep(5)

#######################################
#######################################
# Load other functions from ColabFold #
#######################################
#######################################
import json
import logging
import warnings
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, TYPE_CHECKING

from absl import logging as absl_logging
from importlib_metadata import distribution
from tqdm import TqdmExperimentalWarning



# parse_fasta from colabfold.batch
def parse_fasta(fasta_string: str) -> Tuple[List[str], List[str]]:
    """Parses FASTA string and returns list of strings with amino-acid sequences.
    Arguments:
      fasta_string: The string contents of a FASTA file.
    Returns:
      A tuple of two lists:
      * A list of sequences.
      * A list of sequence descriptions taken from the comment lines. In the
        same order as the sequences.
    """
    sequences = []
    descriptions = []
    index = -1
    for line in fasta_string.splitlines():
        line = line.strip()
        if line.startswith("#"):
            continue
        if line.startswith(">"):
            index += 1
            descriptions.append(line[1:])  # Remove the '>' at the beginning.
            sequences.append("")
            continue
        elif not line:
            continue  # Skip blank lines.
        sequences[index] += line

    return sequences, descriptions

# get_queries from colabfold.batch
def get_queries(
    input_path: Union[str, Path], sort_queries_by: str = "length"
) -> Tuple[List[Tuple[str, str, Optional[List[str]]]], bool]:
    """Reads a directory of fasta files, a single fasta file or a csv file and returns a tuple
    of job name, sequence and the optional a3m lines"""

    input_path = Path(input_path)
    if not input_path.exists():
        raise OSError(f"{input_path} could not be found")

    if input_path.is_file():
        if input_path.suffix == ".csv" or input_path.suffix == ".tsv":
            sep = "\t" if input_path.suffix == ".tsv" else ","
            df = pandas.read_csv(input_path, sep=sep)
            assert "id" in df.columns and "sequence" in df.columns
            queries = [
                (seq_id, sequence.upper().split(":"), None)
                for seq_id, sequence in df[["id", "sequence"]].itertuples(index=False)
            ]
            for i in range(len(queries)):
                if len(queries[i][1]) == 1:
                    queries[i] = (queries[i][0], queries[i][1][0], None)
        elif input_path.suffix == ".a3m":
            (seqs, header) = parse_fasta(input_path.read_text())
            if len(seqs) == 0:
                raise ValueError(f"{input_path} is empty")
            query_sequence = seqs[0]
            # Use a list so we can easily extend this to multiple msas later
            a3m_lines = [input_path.read_text()]
            queries = [(input_path.stem, query_sequence, a3m_lines)]
        elif input_path.suffix in [".fasta", ".faa", ".fa"]:
            (sequences, headers) = parse_fasta(input_path.read_text())
            queries = []
            for sequence, header in zip(sequences, headers):
                sequence = sequence.upper()
                if sequence.count(":") == 0:
                    # Single sequence
                    queries.append((header, sequence, None))
                else:
                    # Complex mode
                    queries.append((header, sequence.upper().split(":"), None))
        else:
            raise ValueError(f"Unknown file format {input_path.suffix}")
    else:
        assert input_path.is_dir(), "Expected either an input file or a input directory"
        queries = []
        for file in sorted(input_path.iterdir()):
            #troubleshooting
            print("Parsing fasta file {}".format(file))

            if not file.is_file():
                continue
            if file.suffix.lower() not in [".a3m", ".fasta", ".faa"]:
                logger.warning(f"non-fasta/a3m file in input directory: {file}")
                continue
            (seqs, header) = parse_fasta(file.read_text())
            if len(seqs) == 0:
                logger.error(f"{file} is empty")
                continue
            query_sequence = seqs[0]
            if len(seqs) > 1 and file.suffix in [".fasta", ".faa", ".fa"]:
                logger.warning(
                    f"More than one sequence in {file}, ignoring all but the first sequence"
                )

            if file.suffix.lower() == ".a3m":
                a3m_lines = [file.read_text()]
                queries.append((file.stem, query_sequence.upper(), a3m_lines))
            else:
                if query_sequence.count(":") == 0:
                    # Single sequence
                    queries.append((file.stem, query_sequence, None))
                else:
                    # Complex mode
                    queries.append((file.stem, query_sequence.upper().split(":"), None))

    # sort by seq. len
    if sort_queries_by == "length":
        queries.sort(key=lambda t: len(t[1]))
    elif sort_queries_by == "random":
        random.shuffle(queries)
    is_complex = False
    for job_number, (raw_jobname, query_sequence, a3m_lines) in enumerate(queries):
        if isinstance(query_sequence, list):
            is_complex = True
            break
        if a3m_lines is not None and a3m_lines[0].startswith("#"):
            a3m_line = a3m_lines[0].splitlines()[0]
            tab_sep_entries = a3m_line[1:].split("\t")
            if len(tab_sep_entries) == 2:
                query_seq_len = tab_sep_entries[0].split(",")
                query_seq_len = list(map(int, query_seq_len))
                query_seqs_cardinality = tab_sep_entries[1].split(",")
                query_seqs_cardinality = list(map(int, query_seqs_cardinality))
                is_single_protein = (
                    True
                    if len(query_seq_len) == 1 and query_seqs_cardinality[0] == 1
                    else False
                )
                if not is_single_protein:
                    is_complex = True
                    break
    return queries, is_complex

# TqdmHandler from colabfold.utils (needed for setup_logging)
class TqdmHandler(logging.StreamHandler):
    """https://stackoverflow.com/a/38895482/3549270"""

    def __init__(self):
        logging.StreamHandler.__init__(self)

    def emit(self, record):
        # We need the native tqdm here
        from tqdm import tqdm

        msg = self.format(record)
        tqdm.write(msg)

# setup_logging from colabfold.utils
def setup_logging(log_file: Path):
    log_file.parent.mkdir(exist_ok=True, parents=True)
    root = logging.getLogger()
    if root.handlers:
        for handler in root.handlers:
            root.removeHandler(handler)
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s %(message)s",
        handlers=[TqdmHandler(), logging.FileHandler(log_file)],
    )
    # otherwise jax will tell us about its search for devices
    absl_logging.set_verbosity("error")
    warnings.simplefilter(action="ignore", category=TqdmExperimentalWarning)

#from colabfold.utils
def safe_filename(file: str) -> str:
    return "".join([c if c.isalnum() or c in ["_", ".", "-"] else "_" for c in file])

# default_data_dir from colabfold.
import appdirs
default_data_dir = Path(appdirs.user_cache_dir(__package__ or "colabfold"))

In [ ]:
#@title ##### 2B.2 Load input fastas
%%time
from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np
from jax.tree_util import tree_map
import matplotlib.pyplot as plt
from scipy.special import softmax

def parse_output(output):
  pae = (output["aligned_confidence_probs"][0] * np.arange(64)).mean(-1) * 31
  plddt = output["plddt"][0,:,1]
  
  bins = np.append(0,np.linspace(2.3125,21.6875,63))
  sm_contacts = softmax(output["distogram_logits"],-1)[0]
  sm_contacts = sm_contacts[...,bins<8].sum(-1)
  xyz = output["positions"][-1,0,:,1]
  mask = output["atom37_atom_exists"][0,:,1] == 1
  o = {"pae":pae[mask,:][:,mask],
       "plddt":plddt[mask],
       "sm_contacts":sm_contacts[mask,:][:,mask],
       "xyz":xyz[mask]}
  if "contacts" in output["lm_output"]:
    lm_contacts = output["lm_output"]["contacts"].astype(float)[0]
    o["lm_contacts"] = lm_contacts[mask,:][:,mask]
  return o

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)


#################
### Load data ###
#################

#@title Input protein sequence, then hit `Runtime` -> `Run all`

#input_dir = '/content/drive/MyDrive/AF2/AAV100_stage_1/stage_1_grouping_2_input_fasta' #@param {type:"string"}
input_dir = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_input_fastas' #@param {type:"string"}

#result_dir = '/content/drive/MyDrive/AF2/ESMFold_AAV100_stage1_results' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_results' #@param {type:"string"}

#Load queries
queries, is_complex = get_queries(input_dir)


# from batch.py
data_dir = default_data_dir
data_dir = Path(data_dir)
result_dir = Path(result_dir)
result_dir.mkdir(exist_ok=True)

# jobname = "test" #@param {type:"string"}
# jobname = re.sub(r'\W+', '', jobname)[:50]

# sequence = "GWSTELEKHREELKEFLKKEGITNVEIRIDNGRLEVRVEGGTERLKRFLEELRQKLEKKGYTVDIKIE" #@param {type:"string"}
# sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
# sequence = re.sub(":+",":",sequence)
# sequence = re.sub("^[:]+","",sequence)
# sequence = re.sub("[:]+$","",sequence)

#@markdown ---
#@markdown #####**Advanced Options**
num_recycles = 3 #@param ["0", "1", "2", "3", "6", "12"] {type:"raw"}

get_LM_contacts = False
#get_LM_contacts = False #@param {type:"boolean"}

copies = 1
# copies = 1 #@param {type:"integer"}

glycine_linker_length = 30 #@param {type:"number"}
# if copies == "" or copies <= 0: copies = 1
# sequence = ":".join([sequence] * copies)

samples = None
masking_rate = 0.15
stochastic_mode = "LM"
# samples = None #@param ["None", "1", "4", "8", "16", "32", "64"] {type:"raw"}
# masking_rate = 0.15 #@param {type:"number"}
# stochastic_mode = "LM" #@param ["LM", "LM_SM", "SM"]

# ID = jobname+"_"+get_hash(sequence)[:5]
# seqs = sequence.split(":")
# lengths = [len(s) for s in seqs]
# length = sum(lengths)
# print("length",length)

# u_seqs = list(set(seqs))
# if len(seqs) == 1: mode = "mono"
# elif len(u_seqs) == 1: mode = "homo"
# else: mode = "hetero"

if "model" not in dir():
  import torch
  model = torch.load("esmfold.model")
  model.cuda().requires_grad_(False)

# # optimized for Tesla T4
# if length > 700:
#   model.trunk.set_chunk_size(64)
# else:
#   model.trunk.set_chunk_size(128)

best_pdb_str = None
best_ptm = 0
best_output = None
traj = []

num_samples = 1 if samples is None else samples





In [ ]:
#@title ##### 2B.3 Run **ESMFold** in batch

###########
## Batch ##
###########


logger = logging.getLogger(__name__)

keep_existing_results = False

if 'logging_setup' not in globals():
    setup_logging(Path(result_dir).joinpath("log.txt"))
    logging_setup = True


for job_number, (raw_jobname, query_sequence, a3m_lines) in enumerate(queries):
    jobname = safe_filename(raw_jobname)
    # In the colab version and with --zip we know we're done when a zip file has been written
    result_zip = result_dir.joinpath(jobname).with_suffix(".result.zip")
    if keep_existing_results and result_zip.is_file():
        logger.info(f"Skipping {jobname} (result.zip)")
        continue
    # In the local version we use a marker file
    is_done_marker = result_dir.joinpath(jobname + ".done.txt")
    if keep_existing_results and is_done_marker.is_file():
        logger.info(f"Skipping {jobname} (already done)")
        continue

    query_sequence_len = (
        len(query_sequence)
        if isinstance(query_sequence, str)
        else sum(len(s) for s in query_sequence)
    )
    logger.info(
        f"Query {job_number + 1}/{len(queries)}: {jobname} (length {query_sequence_len})"
    )


    # #add glycine linker if there isn't one
    glycine_linker_seq = 'G' * glycine_linker_length
    sequence = (
        query_sequence
        if isinstance(query_sequence, str)
        else glycine_linker_seq.join(query_sequence)
    )

    print('> Sequence to model: ')
    print(sequence)

    if len(sequence) > 700:
        model.trunk.set_chunk_size(64)
    else:
        model.trunk.set_chunk_size(128)

    for seed in range(num_samples):
        torch.cuda.empty_cache()
        if samples is None:
            seed = "default"
            mask_rate = 0.0
            model.train(False)
        else:
            torch.manual_seed(seed)
            mask_rate = masking_rate if "LM" in stochastic_mode else 0.0
            model.train("SM" in stochastic_mode)

        output = model.infer(sequence,
                            num_recycles=num_recycles, #deleted argument chain_linker = "X"*chain_linker, from Alphafold-multimer
                            residue_index_offset=512,
                            mask_rate=mask_rate,
                            return_contacts=get_LM_contacts)
        
        pdb_str = model.output_to_pdb(output)[0]
        output = tree_map(lambda x: x.cpu().numpy(), output)
        ptm = output["ptm"][0]
        plddt = output["plddt"][0,:,1].mean()
        traj.append(parse_output(output))
        print(f'{seed} ptm: {ptm:.3f} plddt: {plddt:.1f}')
        if ptm > best_ptm:
            best_pdb_str = pdb_str
            best_ptm = ptm
            best_output = output
        #os.system(f"mkdir -p {ID}")
        if samples is None:
            pdb_filename = result_dir.joinpath(f"{jobname}_unrelaxed_ptm{ptm:.3f}_r{num_recycles}_seed{seed}.pdb")
        else:
            pdb_filename = result_dir.joinpath(f"{jobname}_unrelaxed_ptm{ptm:.3f}_r{num_recycles}_seed{seed}_{stochastic_mode}_m{masking_rate:.2f}.pdb")

        with open(pdb_filename,"w") as out:
            out.write(pdb_str)


In [ ]:
#@title ##### 2B.4 (optional) Terminate runtime
#@markdown The session will be terminated and you will see an "error message".
from google.colab import runtime
runtime.unassign()

### Step 3 - Quantify structure models

In this step, you will use PyMOL script appraise/pymol_quantify_peptide_binding.py to analyze the pdb files in a folder and generate a csv file containing all measurements in parent folder of the pdb results folder. 

*Notes:*

*1. If you did not use ColabFold for the modeling, the file names of the models need to be changed to the following format, where the bracketed part can be any filler string with a total length of 14 characters:*
```
'ReceptorName_and_Peptide1Name_vs_Peptide2Name_vs_..._vs_PeptideNName_unrelaxed_[14Characters].pdb'
```
*2. Currently, Colab-APPRAISE notebook only allows you to run the script with default parameters (for example, the anchor site defaults to the C-terminus). If you need to change the parameters for the quantification function, use Advanced approach below instead.*


In [ ]:
#@markdown #### 3.1 Install open-source PyMOL


#@markdown Hit play button to install. The installation code was adapted from [colabOpenSourcePyMOLpySnips](https://github.com/MooersLab/colabOpenSourcePyMOLpySnips) by Dr. Blaine Mooers.

from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlick's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

Thank you to Professor David Oppenheimer of the University of Florida for suggesting the use mamba and of Open Source PyMOL.
"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install pymol-open-source --yes

        pbar.update(70)

In [ ]:
#@markdown #### 3.2 Run the APPRAISE analysis script

#@markdown **Folder path setting**
folder_path_for_predicted_structures = '/content/drive/MyDrive/APPRAISE_project_1/structure_modeling_results' #@param {type:"string"}
#@markdown - The folder should contain the structure prediction results.


# Get the path to the script
import appraise
import os
script_path = '/'.join(os.path.abspath(appraise.__file__).split('/')[:-2]) + '/appraise/pymol_quantify_peptide_binding.py'

# Execute script
!pymol -cq $script_path $folder_path_for_predicted_structures

You will need to replace "/path/to/APPRAISE" with the actual path. You might also need to change "pymol" to the actual location of the executable, depending on your operation system and PyMOL release.

#### (Advanced) Alternative step 3

You can run the quantification script in a local installation of PyMOL using **PyMOL prompt**, which gives you **more control** of the parameters to be used for analysis. For example, you can change the anchor point of the receptor from the default (C-terminus) to the N-terminus or any other residues.

First, you need a copy of the PyMOL script. If you already have a local installation of [APPRAISE package](https://pypi.org/project/appraise/), the script is also included in the package. If you don't have the package, you can also download the standalone PyMOL script [here](https://github.com/xz-ding/APPRAISE/blob/main/appraise/pymol_quantify_peptide_binding.py). 

Then, in PyMOL GUI, load the script and call the quantify_binding function with appropriate arguments.

For example:

```
# Load the script (replace "/path/to/APPRAISE" with the actual path)
run /path/to/APPRAISE/appraise/pymol_quantify_peptide_binding.py

# Call the quantification function (change the parameters as needed)
quantify_binding('path_to_results_folder/', use_relaxed=False, time_stamp=True, mod_start_resi=3, mod_end_resi=9, pLDDT_threshold=0, membrane_anchor_site='N-term')
```

*Note: The script will take a few minutes to run, and the PyMOL GUI might be frozen while the script is running, which is normal.*

### Step 4 - Analyze quantification results

#### Step 4A - Analyze pairwise competition results

Use this block to analyze pairwise competition results. If you're analyzing pooled competition or single peptide binding results, use block Step 4B instead.

In [ ]:
#@markdown #### 4A.1 Load structure quantification resutls

# Import common packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import necessary APPRAISE modules
import appraise
from appraise.utilities import *
from appraise.score_calculation import *

# Get the settings
#@markdown **Basic settings**

database_path = '/content/drive/MyDrive/APPRAISE_project_1/database_APPRAISE_measurements_pairwise.csv' #@param {type:"string"}
#@markdown - Path to quantification database file (*.csv)
#@markdown - You can find the path in the final lines of the output from step 3.2.



APPRAISE_version = '1.2' #@param ['1.0', '1.1', '1.2'] {type:"string"}
APPRAISE_version = float(APPRAISE_version)
#@markdown - Version of APPRAISE to use

#@markdown **Receptor hydrodynamic properties**

receptor_of_interest = 'Ly6a' #@param {type:"string"}
#@markdown - Receptor of interest (need to match the name in input fasta file names)
receptor_Dmax = 46.68 #@param {type:"number"}
receptor_AxialRatio = 1.74 #@param {type:"number"}
receptor_Rminor = receptor_Dmax/receptor_AxialRatio/2 
#@markdown - You may calculate hydrodynamic properties of your receptor (**receptor_Dmax** and **receptor_AxialRatio**) using the [HullRad server](http://52.14.70.9/Run_hullrad.html). Upload a pdb containing the **receptor only** model to the server and record the **Dmax** and **Axial ratio**. (By default setting, you should have already got 5 predicted receptor-only models in folder_path_for_predicted_structures. Simply upload the model that is ranked #1 to HullRad server.)
#@markdown - Rminor, the hydrodynamic radius along the minor axis of the receptor (considered as an ellipsoid), is automatically calculated using the formula Rminor = Dmax / AxialRatio / 2.



# Read and calculate scores
df = pd.read_csv(database_path)
df = df.loc[df['receptor_name'] == receptor_of_interest].copy()
df['receptor_Rminor'] = receptor_Rminor
list_peptide_names, list_peptide_seqs = get_peptide_list_from_model_names(df)

df_sorted = sort_df_by_peptides_and_cleanup(df, list_peptide_names)
print('\nA database with {} peptides is successfully loaded! \n'.format(len(list_peptide_names)))

# Quality check
database_quality_check(df)

In [ ]:
#@markdown #### 4A.2 Calculate the relative binding scores from each individual competition ($\Delta B_2^{POI, competitor}$):

# Get a dataframe that is sorted pair-wise
df_pairwise_sorted = sort_df_by_peptides_and_cleanup(df, list_peptide_names)

# Calculate the pair-wise scores
df_pairwise_sorted = calculate_scores(df_pairwise_sorted, version=APPRAISE_version, depth_constraint=True)
print('Binding scores from each individual competition:')
df_pairwise_sorted[['peptide_name','competitor','Delta_B']]

In [ ]:
#@markdown #### 4A.3 Get the mean relative binding scores ($\overline{\Delta B}_2^{POI, competitor}$)

# Sort the peptides
df_pairwise_average = df_pairwise_sorted.groupby(by=['peptide_name','competitor','peptide_seq']).mean().dropna(subset=['Delta_B']).reset_index()
print('Mean binding scores:')
df_pairwise_average[['peptide_name','competitor','Delta_B']]

In [ ]:
#@markdown #### 4A.4 Get the heatmap and ranking

#@markdown Run the following code block to get a the absolute binding scores of the variants and a list of top peptides that can be used for stage 2.

#@markdown The results will be saved in the save directory as the database.

#@markdown **Ranking setting**
feature_of_interest = 'Delta_B' #@param ['Delta_B'] {type:"string"}
rank_by_tournament = True #@param {type:"boolean"}
p_value_threshold = 0.05 #@param {type:"number"}
n_competitions = '5+5 (AlphaFold default)' #@param ['5+5 (AlphaFold default)', '1+1 (ESMFold default)'] {type:"string"}
if n_competitions == '5+5 (AlphaFold default)':
    number_of_repeats = 10
if n_competitions == '1+1 (ESMFold default)':
    number_of_repeats = 2


#@markdown **Figure setting**
title='APPRAISE 1.2 ranking' #@param {type:"string"}

auto_range = False #@param {type:"boolean"}
manual_range_vmin = -100 #@param {type:"number"}
manual_range_vmax = 100 #@param {type:"number"}
if auto_range:
    vmin = 'auto'
    vmax = 'auto'
else:
    vmin = manual_range_vmin
    vmax = manual_range_vmax
#@markdown - If auto_range is ON, the manual ranges will be overridden.

auto_fig_size = True #@param {type:"boolean"}
manual_fig_size = 5 #@param {type:"number"}
if auto_fig_size:
    fig_size = 'auto'
else:
    fig_size = manual_fig_size
#@markdown - If auto_fig_size is ON, the manual figure size will be overridden.


list_peptide_order, _, _ = plot_heatmap(df_pairwise_average, feature_of_interest=feature_of_interest, title=rank_by_tournament, rank_by_tournament=rank_by_tournament, save_figure=False, number_of_repeats=number_of_repeats, p_value_threshold=p_value_threshold, fig_size=fig_size, vmin=vmin, vmax=vmax)
heatmap_figure_path = '/'.join(database_path.split('/')[:-1]) + '/APPRAISE_pariwise_results_matrix.png'
plt.savefig(heatmap_figure_path, bbox_inches = 'tight', dpi=300)

print('APPRAISE analysis finished!')
print('The final ranking is: {}'.format(list_peptide_order))
print('The heatmap is saved as {}'.format(heatmap_figure_path))

#### Step 4B - Analyze pooled competition results

Use this block to analyze pooled competition or single peptide binding results. If you're analyzing pairwise competition results, use block Step 4A instead.

In [ ]:
#@markdown #### 4B.1 Load structure quantification resutls

# Import common packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import necessary APPRAISE modules
import appraise
from appraise.utilities import *
from appraise.score_calculation import *

# Get the settings
#@markdown **Basic settings**

database_path = '/content/drive/MyDrive/APPRAISE_project_1/database_APPRAISE_measurements_pooled.csv' #@param {type:"string"}
#@markdown - Path to quantification database file (*.csv)


APPRAISE_version = '1.2' #@param ['1.0', '1.1', '1.2'] {type:"string"}
APPRAISE_version = float(APPRAISE_version)
#@markdown - Version of APPRAISE to use

#@markdown **Receptor properties**

receptor_of_interest = 'Ly6a' #@param {type:"string"}
#@markdown - Name of the receptor of interest (need to match the name in input fasta file names)
receptor_Dmax = 46.68 #@param {type:"number"}
receptor_AxialRatio = 1.74 #@param {type:"number"}
receptor_Rminor = receptor_Dmax/receptor_AxialRatio/2 
#@markdown - You may calculate hydrodynamic properties of your receptor (**receptor_Dmax** and **receptor_AxialRatio**) using the [HullRad server](http://52.14.70.9/Run_hullrad.html). Upload a pdb containing the **receptor only** model to the server and record the **Dmax** and **Axial ratio**. (By default setting, you should have already got 5 predicted receptor-only models in folder_path_for_predicted_structures. Simply upload the model that is ranked #1 to HullRad server.)
#@markdown - Rminor, the hydrodynamic radius along the minor axis of the receptor (considered as an ellipsoid), is automatically calculated using the formula Rminor = Dmax / AxialRatio / 2.



# Read and calculate scores
df = pd.read_csv(database_path)
df = df.loc[df['receptor_name'] == receptor_of_interest].copy()
df['receptor_Rminor'] = receptor_Rminor
list_peptide_names, list_peptide_seqs = get_peptide_list_from_model_names(df)

df_sorted = sort_df_by_peptides_and_cleanup(df, list_peptide_names, consider_competitor_name=False)

print('\nA database with {} peptides is successfully loaded! \n'.format(len(list_peptide_names)))

# Quality check
database_quality_check(df_sorted)

In [ ]:
#@markdown #### 4B.2 Calculate absolute binding scores from each individual competition ($B_2^{POI, competitor}$)

df = calculate_scores(df, version=APPRAISE_version)
print('Binding scores from individual competitions:')
df[['peptide_name','B_POI']]

In [ ]:
#@markdown #### 4B.3 Get the mean binding scores ($\overline{B}_2^{POI}$)
df_average_by_POI = df.groupby(by=['peptide_name','peptide_seq']).mean().dropna(subset=['B_POI']).reset_index()
print('Mean binding scores of each POI:')
df_average_by_POI[['peptide_name','B_POI']]

In [ ]:
#@markdown #### 4B.4 Get a list of top peptides that can be used for stage 2 of HT-APPRAISE.

#@markdown The results will be saved in the save directory as the database.

# Select the top variants
N_top = 18 #@param {type:"integer"}
#@markdown - Number of top variants to keep in the list.
df_average_by_POI = df_average_by_POI.sort_values(by='B_POI', ascending=False).reset_index()
df_selected_peptides = df_average_by_POI.loc[0:N_top-1, ['peptide_name', 'peptide_seq', 'B_POI']]


#Save the results
print('\nSuccess!')
score_file_path = '/'.join(database_path.split('/')[:-1]) + '/APPRAISE{}_scores_of_all_peptides.csv'.format(str(APPRAISE_version))
df_average_by_POI.to_csv(score_file_path)
print('\nThe APPRAISE scores of peptide variants are saved in {}'.format(score_file_path))
selected_peptide_list_path = '/'.join(database_path.split('/')[:-1]) + '/APPRAISE{}_selected_top_{}_peptides.csv'.format(str(APPRAISE_version), str(N_top))
df_selected_peptides.to_csv(selected_peptide_list_path)
print('\nThe list of selected peptides is saved as {}'.format(selected_peptide_list_path))

# Display the results
print('\nHere is the list of selected peptide variants:\n')
df_selected_peptides

In [ ]:
#@markdown #### 4B.5 Get an overview plot

# Configure the plot
sns.set_style("white")
sns.set_context("paper")

total_number_of_variants = len(df_average_by_POI)

#@markdown **Plot setting**

# Set number of subplots
n_rows = 5 #@param {type:"integer"}
n_columns = int((total_number_of_variants + n_rows - 1) / n_rows)
#@markdown - Number of rows of subplots. 


#set figure size
auto_fig_size = True #@param {type:"boolean"}
manual_row_len = 10 #@param {type:"number"}
manual_column_len = 8 #@param {type:"number"}
if auto_fig_size:
    row_len = n_rows * 2
    col_len = n_columns * 0.4
else:
    row_len = manual_row_len
    col_len = manual_column_len
plt.rcParams["figure.figsize"] = (row_len, col_len)
#@markdown - If auto_fig_size is ON, the manual figure sizes will be overridden.

save_figure = True #@param {type:"boolean"}



fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns)

# Plot each peptide in a subplot
for i, peptide_name in enumerate(df_average_by_POI['peptide_name']):
    j = i // n_columns
    k = i % n_columns
    
    # Set the y axis display rules depending on the position
    if k == 0:
        sns.despine(bottom = True, left=False, right=True, ax=axes[j, k])
    else:
        sns.despine(bottom = True, left=True, right=True, ax=axes[j, k])
        axes[j,k].set(yticklabels=[])
 
    # Set coloring -- highlight PHP.B, Dis90, and AAV9
    if peptide_name == 'PHP.B':
        color_setting = sns.color_palette()[0]
    elif peptide_name == 'Dis90':
        color_setting = sns.color_palette()[2]
    elif peptide_name == 'AAV9':
        color_setting = sns.color_palette("tab10")[7]
    else:
        color_setting = sns.color_palette()[7]
    
    # Plot the data points
    sns.stripplot(ax=axes[j,k], y="B_POI", data=df.loc[df['peptide_name']==peptide_name], color=color_setting)
    
    # Plot the mean bar
    sns.boxplot(ax=axes[j,k], 
            showmeans=True,
            meanline=True,
            meanprops={'color': 'k', 'ls': '-', 'lw': 2},
            medianprops={'visible': False},
            whiskerprops={'visible': False},
            zorder=10,
            y="B_POI",
            data=df.loc[df['peptide_name']==peptide_name],
            showfliers=False,
            showbox=False,
            showcaps=False)
    
    # Annotate the reads
    axes[j, k].annotate(int(np.mean(df.loc[df['peptide_name']==peptide_name, "B_POI"])), xy=(0.5, -0.1), xycoords='axes fraction', ha='center', va='center')
    #axes[j, k].set_title(variable_name, wrap=True)
    axes[j,k].set_ylim(-50, 400)
    axes[j,k].set_xlabel('')
    axes[j,k].set_ylabel('')
    axes[j,k].set(xticklabels=[])
   
    
plt.tight_layout()
if save_figure:
    fig.savefig('/'.join(database_path.split('/')[:-1]) + '/APPRAISE_pooled_overview.png', dpi=300)
